In [1]:
import pandas as pd
import numpy as np

def compute_iou(boxA, boxB):
    """Compute IoU between two boxes in MOT format (x, y, w, h)"""
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[0] + boxA[2], boxB[0] + boxB[2])
    yB = min(boxA[1] + boxA[3], boxB[1] + boxB[3])
    
    interW = max(0, xB - xA)
    interH = max(0, yB - yA)
    interArea = interW * interH

    boxAArea = boxA[2] * boxA[3]
    boxBArea = boxB[2] * boxB[3]
    
    unionArea = boxAArea + boxBArea - interArea
    if unionArea == 0:
        return 0
    return interArea / unionArea

# Load MOT-format data (columns: frame, id, x, y, w, h, ... ignored rest)
gt = pd.read_csv("/Users/eliu/Documents/benchmark_eval/data/gt/mot_challenge/MOT17-train/simple_mid/gt/gt.txt", header=None)
trk = pd.read_csv("/Users/eliu/Documents/benchmark_eval/data/trackers/mot_challenge/MOT17-train/YOLOTracker/simple_mid.txt", header=None)

# Only keep necessary columns
gt = gt.iloc[:, :6]
trk = trk.iloc[:, :6]
gt.columns = trk.columns = ["frame", "id", "x", "y", "w", "h"]

# Set IoU threshold
iou_threshold = 0.5

# Check IoUs frame by frame
for frame in sorted(gt["frame"].unique()):
    gt_boxes = gt[gt["frame"] == frame].reset_index(drop=True)
    trk_boxes = trk[trk["frame"] == frame].reset_index(drop=True)

    for i, gt_row in gt_boxes.iterrows():
        gt_box = gt_row[["x", "y", "w", "h"]].values
        for j, trk_row in trk_boxes.iterrows():
            trk_box = trk_row[["x", "y", "w", "h"]].values
            iou = compute_iou(gt_box, trk_box)
            if iou >= iou_threshold:
                print(f"Frame {frame}: GT ID {gt_row['id']} overlaps with Tracker ID {trk_row['id']} (IoU={iou:.3f})")

In [4]:
#visualize 
import os
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import glob

# --- CONFIG ---
gt_file = '/Users/eliu/Documents/benchmark_eval/data/gt/mot_challenge/MOT17-train/simple_mid/gt/gt.txt'
tracker_file = '/Users/eliu/Documents/benchmark_eval/data/trackers/mot_challenge/MOT17-train/YOLOTracker/simple_mid.txt'
frames_folder = '/Users/eliu/Documents/benchmark_eval/data/gt/mot_challenge/MOT17-train/simple_mid/img1'  # Path to folder of images (e.g. 000001.jpg)
frame_ids_to_plot = [1, 2, 3] 
output_dir = '/Users/eliu/Documents/simple_mid_gt'  # Folder to save visualizations
os.makedirs(output_dir, exist_ok=True)

# --- LOAD DATA ---
cols = ['frame', 'id', 'x', 'y', 'w', 'h', 'conf', 'class', 'vis']
gt_df = pd.read_csv(gt_file, header=None, names=cols)
trk_df = pd.read_csv(tracker_file, header=None, names=cols)

# --- VISUALIZE ---
for frame_id in frame_ids_to_plot:
    # Load image
    matching_imgs = glob.glob(os.path.join(frames_folder, f"*_{frame_id}.jpg"))
    if not matching_imgs:
        print(f"No image found for frame {frame_id}")
        continue
    img_path = matching_imgs[0]

    img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)

    # Create plot
    fig, ax = plt.subplots(1, figsize=(12, 8))
    ax.imshow(img)
    ax.set_title(f"Frame {frame_id}")

    # Plot GT boxes (green)
    gt_boxes = gt_df[gt_df['frame'] == frame_id]
    for _, row in gt_boxes.iterrows():
        rect = patches.Rectangle((row['x'], row['y']), row['w'], row['h'],
                                 linewidth=2, edgecolor='green', facecolor='none')
        ax.add_patch(rect)
        ax.text(row['x'], row['y'] - 5, f"GT ID {int(row['id'])}", color='green', fontsize=10)

    # Plot Tracker boxes (red)
    trk_boxes = trk_df[trk_df['frame'] == frame_id]
    for _, row in trk_boxes.iterrows():
        rect = patches.Rectangle((row['x'], row['y']), row['w'], row['h'],
                                 linewidth=2, edgecolor='red', facecolor='none', linestyle='--')
        ax.add_patch(rect)
        ax.text(row['x'], row['y'] + row['h'] + 10, f"TR ID {int(row['id'])}", color='red', fontsize=10)

    # Save or show
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f"frame_{frame_id:06d}.png"))
    plt.close()
    print(f"Saved visualization for frame {frame_id}")

Saved visualization for frame 1
Saved visualization for frame 2
Saved visualization for frame 3
